<a href="https://colab.research.google.com/github/Yihua0906/Credit-Card-Fraud-Detection-2/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
# 統計圖表
import seaborn as sns
# 從 Scikit-learn 導入 t-SNE，一種非線性降維技術，用於數據可視化
from sklearn.manifold import TSNE
# 從 Scikit-learn 導入 PCA 和 TruncatedSVD，用於線性降維和特徵提取
from sklearn.decomposition import PCA, TruncatedSVD
# 導入 Matplotlib 的 patches 模組，用於在圖表中添加自定義圖例或標記
import matplotlib.patches as mpatches
import time
import joblib
import os
# CLassifer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 導入 collections 模組，提供數據結構（如 Counter）用於統計類別分佈
import collections

# 其他工具庫
from sklearn.model_selection import train_test_split

# 導入 make_pipeline，用於創建機器學習流水線，組合預處理和分類步驟
from sklearn.pipeline import make_pipeline

# 從 imblearn 導入專為不平衡數據設計的流水線
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline

# 導入 SMOTE，一種過採樣技術，生成少數類的合成樣本以平衡數據
from imblearn.over_sampling import SMOTE

# 導入 NearMiss，一種欠採樣技術，減少多數類樣本以平衡數據
from imblearn.under_sampling import NearMiss

# 導入 classification_report_imbalanced，用於生成不平衡數據的分類報告
from imblearn.metrics import classification_report_imbalanced

# 評估指標
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report

# 從 collections 導入 Counter，用於統計數據集中類別的數量
from collections import Counter

# 交叉驗證，後者保持類別比例
from sklearn.model_selection import KFold, StratifiedKFold

# 導入 warnings 模組並設置忽略警告訊息，提升代碼可讀性
import warnings
warnings.filterwarnings("ignore")

# Google Drive 整合
try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# 掛載 Google Drive
if IN_COLAB:
    drive.mount('/content/drive')
    # 設定保存路徑，請根據您的實際情況調整
    save_path = '/content/drive/MyDrive/專案資料集/信用卡詐騙'
    # 確保目錄存在
    os.makedirs(save_path, exist_ok=True)
else:
    # 如果不在 Colab 中運行，則使用當前目錄
    save_path = './'
    print("未在 Google Colab 環境中運行，將保存到本地目錄。")

print(f"模型和報告將保存到: {save_path}")

# 讀取信用卡詐欺數據集
df = pd.read_csv('/content/drive/My Drive/專案資料集/信用卡詐騙/creditcard.csv')
df.head()


Mounted at /content/drive
模型和報告將保存到: /content/drive/MyDrive/專案資料集/信用卡詐騙


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [2]:
df.describe()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [3]:
# 檢查是否有缺失值
print("檢查每一欄的缺失值數量：")
print(df.isnull().sum())

# 檢查是否有任何缺失值（總結）
print("\n是否有任何缺失值：", df.isnull().any().any())

檢查每一欄的缺失值數量：
Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

是否有任何缺失值： False


In [ ]:
# 找出包含缺失值的行
rows_with_missing = df[df.isnull().any(axis=1)]
print("包含缺失值的行：")
print(rows_with_missing)

包含缺失值的行：
Empty DataFrame
Columns: [Time, V1, V2, V3, V4, V5, V6, V7, V8, V9, V10, V11, V12, V13, V14, V15, V16, V17, V18, V19, V20, V21, V22, V23, V24, V25, V26, V27, V28, Amount, Class]
Index: []

[0 rows x 31 columns]


In [ ]:
# 刪除含缺失值的行
df_cleaned = df.dropna()

# 檢查結果
print("清理後的數據形狀：", df_cleaned.shape)
print("清理後的缺失值數量：")
print(df_cleaned.isnull().sum())

清理後的數據形狀： (284807, 31)
清理後的缺失值數量：
Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64


In [4]:
# The classes are heavily skewed we need to solve this issue later.
print('No Frauds', round(df['Class'].value_counts()[0]/len(df) * 100,2), '% of the dataset')
print('Frauds', round(df['Class'].value_counts()[1]/len(df) * 100,2), '% of the dataset')

No Frauds 99.83 % of the dataset
Frauds 0.17 % of the dataset


標準化


In [ ]:
from sklearn.preprocessing import RobustScaler
rob_scaler = RobustScaler()

# 標準化 Amount 和 Time，創建新欄位
df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1, 1))
df['scaled_time'] = rob_scaler.fit_transform(df['Time'].values.reshape(-1, 1))

# 刪除原始的 Amount 和 Time 欄位
df = df.drop(['Amount', 'Time'], axis=1)

# 將 scaled_amount 和 scaled_time 移到前面
scaled_amount = df['scaled_amount']
scaled_time = df['scaled_time']
df = df.drop(['scaled_amount', 'scaled_time'], axis=1)
df.insert(0, 'scaled_amount', scaled_amount)
df.insert(1, 'scaled_time', scaled_time)
# 檢查標準化後的前幾行
print("標準化後的前 5 行：")
print(df.head())

標準化後的前 5 行：
   scaled_amount  scaled_time        V1        V2        V3        V4  \
0       1.783274    -0.994983 -1.359807 -0.072781  2.536347  1.378155   
1      -0.269825    -0.994983  1.191857  0.266151  0.166480  0.448154   
2       4.983721    -0.994972 -1.358354 -1.340163  1.773209  0.379780   
3       1.418291    -0.994972 -0.966272 -0.185226  1.792993 -0.863291   
4       0.670579    -0.994960 -1.158233  0.877737  1.548718  0.403034   

         V5        V6        V7        V8  ...       V20       V21       V22  \
0 -0.338321  0.462388  0.239599  0.098698  ...  0.251412 -0.018307  0.277838   
1  0.060018 -0.082361 -0.078803  0.085102  ... -0.069083 -0.225775 -0.638672   
2 -0.503198  1.800499  0.791461  0.247676  ...  0.524980  0.247998  0.771679   
3 -0.010309  1.247203  0.237609  0.377436  ... -0.208038 -0.108300  0.005274   
4 -0.407193  0.095921  0.592941 -0.270533  ...  0.408542 -0.009431  0.798278   

        V23       V24       V25       V26       V27       V28  Class

**透過SMOTE平衡數據集**


In [5]:
# 分離特徵 (X) 和目標變數 (y)
X = df.drop('Class', axis=1)  # 特徵：包含 scaled_amount, scaled_time, V1-V28
y = df['Class']               # 目標：Class 欄位 (0 或 1)

# 顯示原始數據的分佈
print("\n原始數據的分佈：", Counter(y))

# 初始化 SMOTE，設置採樣策略為 1:5 （16.67% 詐欺，83.33% 非詐欺）
smote = SMOTE(sampling_strategy=0.2, random_state=42)

# 應用 SMOTE 生成新數據
X_resampled, y_resampled = smote.fit_resample(X, y)

# 將結果轉回 DataFrame
df_resampled = pd.concat([pd.DataFrame(X_resampled, columns=X.columns),
                          pd.DataFrame(y_resampled, columns=['Class'])], axis=1)

# --- 步驟 3：驗證結果 ---
# 顯示新數據的分佈
print("SMOTE 後的分佈：", Counter(y_resampled))
print("新數據集的形狀：", df_resampled.shape)

# 檢查新數據集的前幾行
print("\n新數據集的前 5 行：")
print(df_resampled.head())

# 檢查新數據集的形狀
print("新數據集的形狀：", df_resampled.shape)


原始數據的分佈： Counter({0: 284315, 1: 492})
SMOTE 後的分佈： Counter({0: 284315, 1: 56863})
新數據集的形狀： (341178, 31)

新數據集的前 5 行：
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376

In [6]:
X = df.drop('Class', axis=1)
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
import threading

def connect_button():
    print("Connect button pressed")
    threading.Timer(300.0, connect_button).start()  # 每秒執行一次

connect_button()  # 啟動定時執行


Connect button pressed


In [ ]:

from sklearn.model_selection import cross_val_score
# --- 步驟 1：訓練隨機森林模型 ---
print("\n訓練隨機森林模型...")
# 創建模型
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
# 訓練模型
rf_model.fit(X_train, y_train)

# --- 步驟 2：評估模型 ---
# 預測測試集
y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

# 預測訓練集 (用於檢測過擬合)
y_train_pred = rf_model.predict(X_train)

# 計算測試集評估指標
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# 計算訓練集評估指標 (用於檢測過擬合)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)

# 計算差異百分比
accuracy_diff = (train_accuracy - accuracy) / train_accuracy * 100 if train_accuracy > 0 else 0
f1_diff = (train_f1 - f1) / train_f1 * 100 if train_f1 > 0 else 0

# 進行交叉驗證
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='f1')
cv_mean = np.mean(cv_scores)
cv_std = np.std(cv_scores)

# 判斷過擬合狀態
overfitting_score = 0  # 0-10分，分數越高越可能過擬合

# 基於準確率差異評分
if accuracy_diff > 20:  # 差異超過20%
    overfitting_score += 4
elif accuracy_diff > 10:  # 差異超過10%
    overfitting_score += 2
elif accuracy_diff > 5:  # 差異超過5%
    overfitting_score += 1

# 基於F1分數差異評分
if f1_diff > 20:
    overfitting_score += 4
elif f1_diff > 10:
    overfitting_score += 2
elif f1_diff > 5:
    overfitting_score += 1

# 基於交叉驗證與測試集表現差異
if abs(cv_mean - f1) > 0.1:
    overfitting_score += 2

# 結果解釋
if overfitting_score >= 7:
    overfitting_status = "嚴重過擬合"
elif overfitting_score >= 4:
    overfitting_status = "中度過擬合"
elif overfitting_score >= 2:
    overfitting_status = "輕微過擬合"
else:
    overfitting_status = "無明顯過擬合"

# 顯示評估結果
print("\n隨機森林模型評估結果（測試集）：")
print(f"準確率 (Accuracy): {accuracy:.4f}")
print(f"精確率 (Precision): {precision:.4f}")
print(f"召回率 (Recall): {recall:.4f}")
print(f"F1 分數: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print("\n詳細分類報告：")
print(classification_report(y_test, y_pred))

# 顯示過擬合分析
print(f"\n過擬合分析:")
print(f"訓練集準確率: {train_accuracy:.4f}, 測試集準確率: {accuracy:.4f}, 差異: {accuracy_diff:.2f}%")
print(f"訓練集F1分數: {train_f1:.4f}, 測試集F1分數: {f1:.4f}, 差異: {f1_diff:.2f}%")
print(f"交叉驗證F1分數 (均值±標準差): {cv_mean:.4f}±{cv_std:.4f}")
print(f"過擬合狀態: {overfitting_status} (分數: {overfitting_score}/10)")

# --- 步驟 3：儲存模型到雲端 ---
model_filename = os.path.join(save_path, 'random_forest_model.joblib')
joblib.dump(rf_model, model_filename)
print(f"\n模型已儲存至雲端：{model_filename}")

# --- 步驟 4：創建並儲存評估報告 ---
# 創建評估結果字典
evaluation_results = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC-AUC',
               'Train Accuracy', 'Train F1 Score', 'Accuracy Diff (%)', 'F1 Diff (%)',
               'CV F1 Mean', 'CV F1 Std', 'Overfitting Score', 'Overfitting Status'],
    'Value': [accuracy, precision, recall, f1, roc_auc,
              train_accuracy, train_f1, accuracy_diff, f1_diff,
              cv_mean, cv_std, overfitting_score, overfitting_status]
}

# 將評估結果轉換為DataFrame並儲存
eval_df = pd.DataFrame(evaluation_results)
eval_csv_filename = os.path.join(save_path, 'random_forest_evaluation.csv')
eval_df.to_csv(eval_csv_filename, index=False)
print(f"評估報告已儲存至雲端：{eval_csv_filename}")

# 儲存測試集預測結果
test_results = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred,
    'Probability': y_pred_proba
})
predictions_csv_filename = os.path.join(save_path, 'random_forest_predictions.csv')
test_results.to_csv(predictions_csv_filename, index=False)
print(f"預測結果已儲存至雲端：{predictions_csv_filename}")

# --- 步驟 5：儲存特徵重要性 ---
# 獲取特徵重要性
feature_importances = pd.DataFrame({
    'Feature': X_train.columns if hasattr(X_train, 'columns') else [f'Feature_{i}' for i in range(X_train.shape[1])],
    'Importance': rf_model.feature_importances_
})
# 按重要性排序
feature_importances = feature_importances.sort_values('Importance', ascending=False)
# 儲存特徵重要性
importances_csv_filename = os.path.join(save_path, 'random_forest_feature_importances.csv')
feature_importances.to_csv(importances_csv_filename, index=False)
print(f"特徵重要性已儲存至雲端：{importances_csv_filename}")

# 創建一個綜合報告文本檔案
with open(os.path.join(save_path, 'random_forest_report.txt'), 'w', encoding='utf-8') as f:
    f.write("隨機森林模型評估報告\n")
    f.write("========================\n\n")
    f.write(f"準確率 (Accuracy): {accuracy:.4f}\n")
    f.write(f"精確率 (Precision): {precision:.4f}\n")
    f.write(f"召回率 (Recall): {recall:.4f}\n")
    f.write(f"F1 分數: {f1:.4f}\n")
    f.write(f"ROC-AUC: {roc_auc:.4f}\n\n")
    f.write("詳細分類報告：\n")
    f.write(classification_report(y_test, y_pred))
    f.write("\n過擬合分析:\n")
    f.write(f"訓練集準確率: {train_accuracy:.4f}, 測試集準確率: {accuracy:.4f}, 差異: {accuracy_diff:.2f}%\n")
    f.write(f"訓練集F1分數: {train_f1:.4f}, 測試集F1分數: {f1:.4f}, 差異: {f1_diff:.2f}%\n")
    f.write(f"交叉驗證F1分數 (均值±標準差): {cv_mean:.4f}±{cv_std:.4f}\n")
    f.write(f"過擬合狀態: {overfitting_status} (分數: {overfitting_score}/10)\n\n")
    f.write("前10個最重要特徵：\n")
    for i, (feature, importance) in enumerate(zip(feature_importances['Feature'].head(10),
                                               feature_importances['Importance'].head(10))):
        f.write(f"{i+1}. {feature}: {importance:.4f}\n")

print(f"綜合報告已儲存至雲端：{os.path.join(save_path, 'random_forest_report.txt')}")
print("\n所有檔案已成功保存到雲端硬碟！")


訓練隨機森林模型...
Connect button pressed
Connect button pressed
Connect button pressed
Connect button pressed

隨機森林模型評估結果（測試集）：
準確率 (Accuracy): 0.9996
精確率 (Precision): 0.9625
召回率 (Recall): 0.7857
F1 分數: 0.8652
ROC-AUC: 0.9579

詳細分類報告：
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.96      0.79      0.87        98

    accuracy                           1.00     56962
   macro avg       0.98      0.89      0.93     56962
weighted avg       1.00      1.00      1.00     56962


過擬合分析:
訓練集準確率: 1.0000, 測試集準確率: 0.9996, 差異: 0.04%
訓練集F1分數: 1.0000, 測試集F1分數: 0.8652, 差異: 13.48%
交叉驗證F1分數 (均值±標準差): 0.8427±0.0199
過擬合狀態: 輕微過擬合 (分數: 2/10)

模型已儲存至雲端：/content/drive/MyDrive/專案資料集/信用卡詐騙/random_forest_model.joblib
評估報告已儲存至雲端：/content/drive/MyDrive/專案資料集/信用卡詐騙/random_forest_evaluation.csv
預測結果已儲存至雲端：/content/drive/MyDrive/專案資料集/信用卡詐騙/random_forest_predictions.csv
特徵重要性已儲存至雲端：/content/drive/MyDrive/專案資料集/信用卡詐騙/random_forest_feature